# Efecto aleatorio

Clase 16 - 11/10/2018 

## Recetas

### Vemos si el efecto aleatorios son significativos

1) Para ver componentes de varianza (si un efecto aleatorio es significativo), solo para ver la parte aleatoria:

- Estimar el modelo con ``lmer()``, ``library(lme4)``. Se usa asi:
    
```
library(lme4)
modelo = lmer(Y~FIJO + (1|Aleatorio1) + (1|Aleatorio2), RML = TRUE)
```

- Usar con la opción ``REML = TRUE`` usa máxima verosimilitud

- Construir intervalo de confianza para la varianza: ``confint(modelo)``. Si contiene el 0 entonces ese término de varianza no es significativo.

- Otra forma es comparando modelos

```
modelo_grande = lmer(Y~1 + (1|Aleatorio1) + (1|Aleatorio2))
modelo_chico = lmer(Y~1 + (1|Aleatorio1))
anova(modelo_grande, modelo_chico) # me va a dar un pvalor que me dice si es significativo el efecto aleatorio 2
```

Este anova usa el test usa chi2 que sólo es válido asintoticamente, eso quiere decir que anda bien para un n grande. Por lo tanto, como esto suele ser costoso, podemos usar (según apunte de la UBA) esto otro:

```
library(RLRsim)
exactLRT(modelo_grande, modelo_chico)
```

### Vemos si el efecto fijo es significativo

Ahora veamos como hacer cuando nos interesa mirar la parte fija. Quiero ver si el efecto fijo es significativo

2) Para efectos fijos (test)
    - Usar ``lmer`` con opción ``REML = FALSE``
    
```
library(lme4)
m1 = lmer(Y~FIJO + (1|Aleatorio1),  RML = FALSE)
m0 = lmer(Y~1 + (1|Aleatorio1),  RML = FALSE)
anova(m1, m0)
```

Sigue siendo un test chi2, asi que tampoco le puedo creer tanto, asi que usamos lo que nos dice el apunte de la UBA para ver qué usar en este caso.

```
library(pbkrtest)
KRmodcomp(m1, m0) # usa otra aproximacion en vez de chi2
PBmodcomp(m1, m0)
```

Veremos el pvalor para saber si es significativo. Si el factor nos da significativo el efecto fijo entonces ahora vamos a hacer **comparaciones múltiples** con ``multcomp``.

```
library(multcomp)
m1 = lmer(Y~FIJO + (1|Aleatorio1),  RML = FALSE)
mc = glht(m1, linfct = mcp(FIJO="Tukey")) # construye contrastes de medias para el factor FIJO usando tukey
# si quisieramos usar dunnet debemos poner 1ro el control

```


## Efectos Anidados

Por ejemplo tenemos 3 fabricantes (Fab1, Fab2, Fab3) y quiero ver la variabilidad de cada uno, asi que le pido a cada fabricante que me preste 5 máquinas (M1, M2, M3, M4, M5). 

- Fab: 1, 2, 3 (efecto fijo)
- Maquina: 1, 2, 3, 4, 5 (anidado a Fab porque cada maquina es de cada fabricante)

El modelo sería:

$$y_{ijk} = \mu + \alpha_i + B_{j(i)} + \epsilon_{k(ij)}$$

donde $\alpha$ es el fabricante y B los efectos anidados. 

Los GL de $\alpha = a-1$ y de $B(\alpha)=a(b-1)$

La tabla ANOVA nos va a servir para saber la potencia, por lo tanto de ahi sacamos los valores esperados. 

```
# asi le decimos la anidacion en R
m = lmer(Y~Fab + (1|Maq%in%Fab))
```

### Hacemos un ejemplo

Ejercicio 3 (pag. 272 Kuehl). Fórmulas de aleación con cuatro colados distintos. 
a) escriba el modelo lineal para el experimento, suponga que las aleaciones son efectos fijos y que los colados dentro de las aleaciones y las barras dentro de ellos 

In [7]:
#/home/emiliano/EstadisticaAplicada/Estadistica.Aplicada.2018/03_efectos_aleatorios_y_mixtos/datos/ej3_ch7.txt
ej3_ch7 = read.csv("/home/emiliano/EstadisticaAplicada/Estadistica.Aplicada.2018/03_efectos_aleatorios_y_mixtos/datos/ej3_ch7.txt", sep=" ")
Alea= c(1,1,2,2,3,3)
Aleacion = as.factor(rep(Alea,4))

Moldes = NULL
Y = NULL
for (j in 1:4){
    Y = c(Y, ej3_ch7[,j])
    Moldes = c(Moldes, rep(j,6))
}
Moldes = as.factor(Moldes)
(cbind(Y,Aleacion, Moldes))

# ojo que hay algo que esta mal porque la tabla no queda bien, revisar.
# probar usando Rstudio a ver como lo carga

Warning message in cbind(Y, Aleacion, Moldes):
“number of rows of result is not a multiple of vector length (arg 1)”

Y,Aleacion,Moldes
15.5,1,1
17.1,1,1
16.7,2,1
14.1,2,1
14.8,3,1
15.0,3,1
16.5,1,2
17.3,1,2
13.2,2,2
13.9,2,2


In [10]:
library(lattice)
library(car)
library(MASS)
library(lme4)

#analisis exploratorio
boxplot(Y~Aleacion)
xyplot(Y~Aleacion)

Loading required package: carData


ERROR: Error in model.frame.default(formula = Y ~ Aleacion): variable lengths differ (found for 'Aleacion')


In [11]:
# que modelo es adecuado
library(RLRsim)

m = lmer(Y~Aleacion + (1|Moldes%in%Aleacion), REML=TRUE)
(confint(m))

print(m)

# veremos que la variabilidad introducida por los moldes no es significativa. 

# otra forma de verlo es hacer

# uso lm que no tiene en cuenta efectos aleatorios, sino me da un error
# vemos el pvalor nos dice que no es significativo
m0 = lm(Y~Aleacion)
anova(m, m1)

m = lmer(Y~Aleacion + (1|Moldes%in%Aleacion), REML=FALSE)

# hacemos las comparaciones multiples sobre el efecto fijo
library(multcomp)
mc = glht(m, linfct=mcp(Aleacion="Tukey"))
plot(confint(mc))


# si hacemos lo mismo pero con el m0 veremos que no cambia mucho
mc0 = glht(m0, linfct=mcp(Aleacion="Tukey"))
plot(confint(mc0))

# vemos que los IC son apenas mas chicos, si la variabilidad es significativa ACHICA mas el intervalo. 